In [2]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline
#random.seed(42)
#torch.manual_seed(42)
#np.random.seed(42)

In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        return x

class CNNModel(nn.Module):
  def __init__(self, time_step,n_layers,vector_length,kernel_size):
    super(CNNModel, self).__init__()
    self.time_step=time_step
    self.n_layers=n_layers
    in_channels=1
    out_channels=16
    layers=[]
    dimension=vector_length
    for l in range(self.n_layers):
        cnn_1d_layer=nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding="same")
        #dimension=dimension-kernel_size+2*1+1
        #if dimension>1:
        layers.append(cnn_1d_layer)
        if l%2==0:
          layers.append(nn.Tanh())
        else:
          layers.append(nn.ELU(inplace=True))
        in_channels=out_channels
        out_channels=out_channels*2
    layers.append(Flatten())
    layers.append(nn.Dropout(p=0.2))
    self.body = nn.Sequential(*layers)
    #print('dm',dimension,out_channels)
    out=int(vector_length*(out_channels/2))
    self.head=nn.Linear(out, 1)
  def forward(self, x):
    b, features, look_back = x.shape
    #print(b,n_steps, features)
    x = x.reshape([b,1,features*look_back])
    y = self.body(x)
    #print(y.shape)
    #print('re',y.view(len(y),-1).shape)
    return self.head(y.view(len(y),-1))

In [ ]:
Shortlisted_States=['Maharashtra','Delhi','Uttar-Pradesh','Kerala','Tamil-Nadu']
results_cnn=[]
for state in Shortlisted_States:
  best_models=[]
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-19"]
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']]
  #print(df.describe())

  time_step=[5,7,15,30]
  Number_of_feature=[1,2,3,4,5,6]
  multi_feature=True
  
  for n_f in Number_of_feature:
    for t_s in time_step:
      train_loader, test_loader,scaler = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8)
      for n_layers in range(2,5,1):
        for kernel_size in range(1,5,1):
          
          max_epochs=25
          #random.seed(42)
          #torch.manual_seed(42)
          #np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
          cuda=torch.cuda.is_available()
          if cuda:
            CNN_model = CNN_model.cuda()
          optimizer = optim.SGD(CNN_model.parameters(), lr=1e-2, momentum=0.9)
          train_losses,test_losses, best_model = fit(CNN_model, optimizer, nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=cuda)
          end = time.time()
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
          CNN_model.load_state_dict(best_model)
          CNN_model.eval()
          test_x,test_y=test_loader
          predictions=CNN_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          #predictions = scaler.inverse_transform(predictions)
          #target = scaler.inverse_transform(target)
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_cnn.append([state,n_f,t_s,n_layers,kernel_size,mae,rmse,r2s])
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,'kernel_size',kernel_size,mae,rmse,r2s)
      #CNN_model =  CNNModel(t_s,n_layers,t_s*n_f,kernel_size)
      #CNN_model.load_state_dict(best_model)

In [ ]:
df_cnn = pd.DataFrame (results_cnn,columns=['State','Number_feature','Time_Step','number_layers','kernel_size','MAE','RMSE','R2_Score'])
df_cnn.head()

In [ ]:
#github_upload(folder_name='Indian-States-Model-Results',file_name='CNN.csv', file_data=df_cnn.to_csv())